# Initialize

In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.5` // Or use any other 2.x version here
import $ivy.`sh.almond::almond-spark:0.9.1` // Not required since almond 0.7.0 (will be automatically added when importing spark)

import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.spark.sql._

//jupyter notebook way to create spark session
val spark = {
  NotebookSparkSession.builder()
    .master("local[*]")
    .getOrCreate()
}

Loading spark-stubs
Getting spark JARs
Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


Spark UI

import $ivy.$                                   // Or use any other 2.x version here

import $ivy.$                               // Not required since almond 0.7.0 (will be automatically added when importing spark)


import org.apache.log4j.{Level, Logger}

import org.apache.spark.sql._

//jupyter notebook way to create spark session

spark: SparkSession = org.apache.spark.sql.SparkSession@10ece09

In [2]:
val sc = spark.sparkContext

sc: org.apache.spark.SparkContext = org.apache.spark.SparkContext@c3555

In [3]:
val sqlContext = new SQLContext(sc)

sqlContext: SQLContext = org.apache.spark.sql.SQLContext@b58d0

In [4]:
import sqlContext.implicits._

import sqlContext.implicits._

# Define case class Flight

In [5]:
case class Flight(
                 date: Flight.Date,  //fields 1-4.
                 times: Flight.Times,//fields 5-8, 12-16, 20-21,
                 uniqueCarrier: String, //9: unique carrier code
                 flightNum: Int,        //10: flight number
                 tailNum: String,       //11: plane tail number
                 origin: String,        //17: origin IATA airport code
                 dest: String,          //18: destination IATA airport code
                 distance: Int,         //19: in miles
                 cancelled: Int,        //22: was flight canceled?
                 cancellationCode: String, //23: reason for cancellation(A=Carrier, B=weather, C=NAS)
                 diverted: Int,         //24: 1=yes, 0=no
                 carrierDelay: Int,     //25: in minutes
                 weatherDelay: Int,     //26: in minutes
                 nasDelay: Int,         //27: in minutes
                 securitDelay: Int,     //28: in minutes
                 lateAircraftDelay: Int //29: in minutes
)

object Flight{
  def parse(str:String):Flight={
    val fields = str.split(",")
    val date: Flight.Date = Flight.Date(Integer.parseInt(fields(0)),Integer.parseInt(fields(1)),Integer.parseInt(fields(2)),Integer.parseInt(fields(3)))
    /*
     case class Times(
                  depTime: Int,             //5: autal departure time(local,hhmm)
                  crsDepTime: Int,          //6: scheduled departure time
                  arrTime: Int,             //7: actual arrival time
                  crsArrTime: Int,          //8: scheduled arrival time
                  actualElapsedTime: Int,   //12: in minutes
                  crsElapsedTime: Int,      //13: in minutes
                  airTime: Int,             //14: in minutes
                  arrDelay: Int,            //15: arrival delay, in minutes
                  depDelay: Int,            //16: departure delay, in minutes
                  taxiIn: Int,              //20: taxi in time, in minutes
                  taxiOut: Int              //21: taxi out time, in minutes
                  ) extends Ordered[Times] {
     */
    val times: Flight.Times = Flight.Times(Integer.parseInt(fields(4)),Integer.parseInt(fields(5))
      ,Integer.parseInt(fields(6)),Integer.parseInt(fields(7)),Integer.parseInt(fields(11)),Integer.parseInt(fields(12))
      ,Integer.parseInt(fields(13)),Integer.parseInt(fields(14)),Integer.parseInt(fields(15))
      ,Integer.parseInt(fields(19)),Integer.parseInt(fields(20)))
    /*
    case class Flight(
                 date: Flight.Date,  //fields 1-4.
                 times: Flight.Times,//fields 5-8, 12-16, 20-21,
                 uniqueCarrier: String, //9: unique carrier code
                 flightNum: Int,        //10: flight number
                 tailNum: String,       //11: plane tail number
                 origin: String,        //17: origin IATA airport code
                 dest: String,          //18: destination IATA airport code
                 distance: Int,         //19: in miles
                 cancelled: Int,        //22: was flight canceled?
                 cancellationCode: Int, //23: reason for cancellation(A=Carrier, B=weather, C=NAS)
                 diverted: Int,         //24: 1=yes, 0=no
                 carrierDelay: Int,     //25: in minutes
                 weatherDelay: Int,     //26: in minutes
                 nasDelay: Int,         //27: in minutes
                 securitDelay: Int,     //28: in minutes
                 lateAircraftDelay: Int //29: in minutes
                 )
     */
    val carrier = fields(8)
    val flightNum = Integer.parseInt(fields(9))
    val tailNum = fields(10)
    val origin = fields(16)
    val dest = fields(17)
    val distance = Integer.parseInt(fields(18))

    val cancelled = Integer.parseInt(fields(21))
    //val cancellationCode = if(fields(22).trim.length != 0) Integer.parseInt(fields(22)) else 0
    val cancellationCode = fields(22)

    val diverted = Integer.parseInt(fields(23))

    val carrierDelay = if(fields(24).trim != "NA") Integer.parseInt(fields(24)) else 0
    val weatherDelay = if(fields(25).trim != "NA") Integer.parseInt(fields(25)) else 0
    val nasDelay = if(fields(26).trim != "NA") Integer.parseInt(fields(26)) else 0
    val securitDelay = if(fields(27).trim != "NA") Integer.parseInt(fields(27)) else 0
    val lateAircraftDelay = if(fields(28).trim != "NA") Integer.parseInt(fields(28)) else 0

    Flight(date,times,carrier,flightNum,tailNum,origin,dest,distance, cancelled,cancellationCode,diverted,carrierDelay,weatherDelay,nasDelay,securitDelay,lateAircraftDelay)
  }
  case class Date(
                 year: Int,  //1: 1987-2008
                 month: Int, //2: 1-12
                 dayOfMonth: Int, //3: 1-31
                 dayOfWeek: Int   //4: 1(Monday) - 7(Sunday)
                 )extends Ordered[Date] {
    override def compare(that: Date):Int={
      val diffYear = this.year - that.year
      if(diffYear != 0 )
        diffYear
      else{
        val diffMonth = month - that.month
        if(diffMonth != 0)
          diffMonth
        else
          dayOfMonth - that.dayOfMonth
      }
    }

    //counts from 1 and starts at Monday
    private val dayOfWeekToString = Vector(
      "","Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"
    )

    override def toString: String = "%4d-%02d-%02d(%s)".format(year,month,dayOfMonth,dayOfWeekToString(dayOfWeek))
  }

  case class Times(
                  depTime: Int,             //5: autal departure time(local,hhmm)
                  crsDepTime: Int,          //6: scheduled departure time
                  arrTime: Int,             //7: actual arrival time
                  crsArrTime: Int,          //8: scheduled arrival time
                  actualElapsedTime: Int,   //12: in minutes
                  crsElapsedTime: Int,      //13: in minutes
                  airTime: Int,             //14: in minutes
                  arrDelay: Int,            //15: arrival delay, in minutes
                  depDelay: Int,            //16: departure delay, in minutes
                  taxiIn: Int,              //20: taxi in time, in minutes
                  taxiOut: Int              //21: taxi out time, in minutes
                  ) extends Ordered[Times] {
    /** only compare the first four fields */
    override def compare(that: Times): Int ={
      val diffDepTime = depTime - that.depTime
      if(diffDepTime != 0) diffDepTime
      else{
        val diffCrsDepTime = crsDepTime - that.crsDepTime
        if(diffCrsDepTime != 0)
          diffCrsDepTime
        else{
          val diffArrTime = arrTime - that.arrTime
          if(diffArrTime != 0)
            diffArrTime
          else
            crsArrTime - that.crsArrTime
        }
      }
    }
  }
}

defined class Flight
defined object Flight

# Test case class Flight

In [6]:
val flight = Flight.parse("2008,1,1,2,2057,2052,2312,2258,AS,234,N306AS,135,126,112,14,5,SEA,SJC,697,7,16,0,,0,NA,NA,NA,NA,NA")
println(flight.flightNum + ";" + flight.date)

234;2008-01-01(Tuesday)


flight: Flight = Flight(
  Date(2008, 1, 1, 2),
  Times(2057, 2052, 2312, 2258, 135, 126, 112, 14, 5, 7, 16),
  "AS",
  234,
  "N306AS",
  "SEA",
  "SJC",
  697,
  0,
  "",
  0,
  0,
  0,
  0,
  0,
  0
)

# Create Flight RDD

In [7]:
val flightTextRDD = sc.textFile("/home/pi/sparktest/data/sparkdataframe/flights.csv")
val header = flightTextRDD.first()
val flightRDD = flightTextRDD.filter(_!=header)

first at cmd6.sc:2

1 / 1

flightTextRDD: org.apache.spark.rdd.RDD[String] = /home/pi/sparktest/data/sparkdataframe/flights.csv MapPartitionsRDD[1] at textFile at cmd6.sc:1
header: String = "1-year,2-month,3-dayOfMonth,4-dayOfWeek,5-ActualDepartureTime,6-ScheduleDepartureTime,7-actualArrivalTime,8-scheduleArrivalTime,9-CarrierCode,10-flightNum,11-planeTailNum,12-actualElapsedTime,13-crsElapsedTime,14-airTime,15-ArrivalDelay,16-DepartureDelay,17-OriginAirportCode,18-DestAirportCode,19-Miles,20-taxiInTime,21-TaxiOutTime,22-flightCanceled?,23-CacellationReason,24-diverted,25-CarrierDelay,26-weatherDelay,27-nasDelay,28-SecurityDelay,29-lateAircraftDelay"
flightRDD: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[2] at filter at cmd6.sc:3

In [8]:
flightRDD.collect()

collect at cmd7.sc:1

2 / 2

res7: Array[String] = Array(
  "2008,1,1,2,2057,2052,2312,2258,AS,234,N306AS,135,126,112,14,5,SEA,SJC,697,7,16,1,A,0,NA,NA,NA,NA,NA",
  "2008,11,1,2,2057,2052,2312,2258,AS,234,N309AS,135,126,112,14,5,SEA,SJC,697,7,16,1,A,0,NA,NA,NA,NA,NA",
  "2008,1,1,2,703,715,958,951,AS,572,N302AS,175,156,144,7,-12,SEA,PSP,987,6,25,0,,0,NA,NA,NA,NA,NA"
)

In [9]:
val flightObjRDD = for{
    line <- flightRDD
    singleFlight = Flight.parse(line)
}yield singleFlight

flightObjRDD: org.apache.spark.rdd.RDD[Flight] = MapPartitionsRDD[4] at map at cmd8.sc:2

In [10]:
flightObjRDD.foreach(println)

foreach at cmd9.sc:1

2 / 2

Flight(2008-01-01(Tuesday),Times(2057,2052,2312,2258,135,126,112,14,5,7,16),AS,234,N306AS,SEA,SJC,697,1,A,0,0,0,0,0,0)
Flight(2008-11-01(Tuesday),Times(2057,2052,2312,2258,135,126,112,14,5,7,16),AS,234,N309AS,SEA,SJC,697,1,A,0,0,0,0,0,0)
Flight(2008-01-01(Tuesday),Times(703,715,958,951,175,156,144,7,-12,6,25),AS,572,N302AS,SEA,PSP,987,0,,0,0,0,0,0,0)


# Define case class Airport

In [11]:
case class Airport(
    iataCode: String,
    airportName: String,
    cityName: String,
    stateName: String,
    countryName: String,
    latitude: Float,
    longitude: Float
)

object Airport{
    def parse(str:String):Airport={
        val fields = str.split(",")
        Airport(fields(0).replace("\"","")
                ,fields(1).replace("\"","")
                ,fields(2).replace("\"","")
                ,fields(3).replace("\"","")
                ,fields(4).replace("\"","")
                ,java.lang.Float.parseFloat(fields(5))
                ,java.lang.Float.parseFloat(fields(6))
            )
    }
}

val a = Airport.parse(""""00M","Thigpen","Bay Springs","MS","USA",31.95376472,-89.23450472""")
println(a.iataCode)

00M


defined class Airport
defined object Airport
a: Airport = Airport(
  "00M",
  "Thigpen",
  "Bay Springs",
  "MS",
  "USA",
  31.953764F,
  -89.234505F
)

# Create Airport RDD

In [12]:
val airportTextRdd = sc.textFile("/home/pi/sparktest/data/sparkdataframe/airports.csv")
val header = airportTextRdd.first()
val airportRDD = airportTextRdd.filter(_!=header)

first at cmd11.sc:2

1 / 1

airportTextRdd: org.apache.spark.rdd.RDD[String] = /home/pi/sparktest/data/sparkdataframe/airports.csv MapPartitionsRDD[6] at textFile at cmd11.sc:1
header: String = "1-\"iata\",2-\"airport\",3-\"city\",4-\"state\",5-\"country\",6-\"lat\",\"7-long\""
airportRDD: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[7] at filter at cmd11.sc:3

In [13]:
val airportObjRDD = for{
    line <- airportRDD
    singleAirport = Airport.parse(line)
}yield singleAirport

airportObjRDD: org.apache.spark.rdd.RDD[Airport] = MapPartitionsRDD[9] at map at cmd12.sc:2

# Create DataFrame

In [14]:
val flights = sqlContext.createDataFrame(flightObjRDD)
val airports = sqlContext.createDataFrame(airportObjRDD)

flights.cache
airports.cache()

flights: DataFrame = [date: struct<year: int, month: int ... 2 more fields>, times: struct<depTime: int, crsDepTime: int ... 9 more fields> ... 14 more fields]
airports: DataFrame = [iataCode: string, airportName: string ... 5 more fields]
res13_2: DataFrame = [date: struct<year: int, month: int ... 2 more fields>, times: struct<depTime: int, crsDepTime: int ... 9 more fields> ... 14 more fields]
res13_3: DataFrame = [iataCode: string, airportName: string ... 5 more fields]

# Using DataFrame API

## API: filter

In [15]:
val canceledFlights = flights.filter(flights("cancelled") > 0)
//explain the query
//canceledFlights.explain(extended=false)
canceledFlights.cache
canceledFlights.show()

show at cmd14.sc:5

1 / 1

show at cmd14.sc:5

1 / 1

+----------------+--------------------+-------------+---------+-------+------+----+--------+---------+----------------+--------+------------+------------+--------+------------+-----------------+
|            date|               times|uniqueCarrier|flightNum|tailNum|origin|dest|distance|cancelled|cancellationCode|diverted|carrierDelay|weatherDelay|nasDelay|securitDelay|lateAircraftDelay|
+----------------+--------------------+-------------+---------+-------+------+----+--------+---------+----------------+--------+------------+------------+--------+------------+-----------------+
| [2008, 1, 1, 2]|[2057, 2052, 2312...|           AS|      234| N306AS|   SEA| SJC|     697|        1|               A|       0|           0|           0|       0|           0|                0|
|[2008, 11, 1, 2]|[2057, 2052, 2312...|           AS|      234| N309AS|   SEA| SJC|     697|        1|               A|       0|           0|           0|       0|           0|                0|
+----------------+-------

canceledFlights: Dataset[Row] = [date: struct<year: int, month: int ... 2 more fields>, times: struct<depTime: int, crsDepTime: int ... 9 more fields> ... 14 more fields]
res14_1: Dataset[Row] = [date: struct<year: int, month: int ... 2 more fields>, times: struct<depTime: int, crsDepTime: int ... 9 more fields> ... 14 more fields]

## API: orderBy

In [16]:
//order by destination airport
flights.orderBy(flights("dest")).show()
//order by desc
flights.orderBy($"dest".desc).show()

show at cmd15.sc:1

2 / 2

+----------------+--------------------+-------------+---------+-------+------+----+--------+---------+----------------+--------+------------+------------+--------+------------+-----------------+
|            date|               times|uniqueCarrier|flightNum|tailNum|origin|dest|distance|cancelled|cancellationCode|diverted|carrierDelay|weatherDelay|nasDelay|securitDelay|lateAircraftDelay|
+----------------+--------------------+-------------+---------+-------+------+----+--------+---------+----------------+--------+------------+------------+--------+------------+-----------------+
| [2008, 1, 1, 2]|[703, 715, 958, 9...|           AS|      572| N302AS|   SEA| PSP|     987|        0|                |       0|           0|           0|       0|           0|                0|
| [2008, 1, 1, 2]|[2057, 2052, 2312...|           AS|      234| N306AS|   SEA| SJC|     697|        1|               A|       0|           0|           0|       0|           0|                0|
|[2008, 11, 1, 2]|[2057, 

show at cmd15.sc:3

2 / 2

+----------------+--------------------+-------------+---------+-------+------+----+--------+---------+----------------+--------+------------+------------+--------+------------+-----------------+
|            date|               times|uniqueCarrier|flightNum|tailNum|origin|dest|distance|cancelled|cancellationCode|diverted|carrierDelay|weatherDelay|nasDelay|securitDelay|lateAircraftDelay|
+----------------+--------------------+-------------+---------+-------+------+----+--------+---------+----------------+--------+------------+------------+--------+------------+-----------------+
|[2008, 11, 1, 2]|[2057, 2052, 2312...|           AS|      234| N309AS|   SEA| SJC|     697|        1|               A|       0|           0|           0|       0|           0|                0|
| [2008, 1, 1, 2]|[2057, 2052, 2312...|           AS|      234| N306AS|   SEA| SJC|     697|        1|               A|       0|           0|           0|       0|           0|                0|
| [2008, 1, 1, 2]|[703, 7

## API: groupBy and count

In [17]:
val canceledFlightsByMonth = canceledFlights.groupBy("date.month").count()
canceledFlightsByMonth.show

show at cmd16.sc:2

2 / 2

show at cmd16.sc:2

1 / 1

show at cmd16.sc:2

...

100 / 100

show at cmd16.sc:2

75 / 75

+-----+-----+
|month|count|
+-----+-----+
|    1|    1|
|   11|    1|
+-----+-----+



canceledFlightsByMonth: DataFrame = [month: int, count: bigint]

## Access DataFrame

In [18]:
flights.getClass
canceledFlights.getClass
canceledFlightsByMonth.getClass

val flightsArr = flights.collect()
val firstRow = flightsArr(0)
firstRow.getClass
firstRow.getString(2)
firstRow(0) //the same as time1.get(0)

collect at cmd17.sc:5

2 / 2

res17_0: Class[T] = class org.apache.spark.sql.Dataset
res17_1: Class[T] = class org.apache.spark.sql.Dataset
res17_2: Class[T] = class org.apache.spark.sql.Dataset
flightsArr: Array[Row] = Array(
  [[2008,1,1,2],[2057,2052,2312,2258,135,126,112,14,5,7,16],AS,234,N306AS,SEA,SJC,697,1,A,0,0,0,0,0,0],
  [[2008,11,1,2],[2057,2052,2312,2258,135,126,112,14,5,7,16],AS,234,N309AS,SEA,SJC,697,1,A,0,0,0,0,0,0],
  [[2008,1,1,2],[703,715,958,951,175,156,144,7,-12,6,25],AS,572,N302AS,SEA,PSP,987,0,,0,0,0,0,0,0]
)
firstRow: Row = [[2008,1,1,2],[2057,2052,2312,2258,135,126,112,14,5,7,16],AS,234,N306AS,SEA,SJC,697,1,A,0,0,0,0,0,0]
res17_5: Class[T] = class org.apache.spark.sql.catalyst.expressions.GenericRowWithSchema
res17_6: String = "AS"
res17_7: Any = [2008,1,1,2]

# DataFrame join

## Using DataFrame API: select

In [21]:
flights.printSchema()
airports.printSchema()
flights.show(10)
airports.show(10)

root
 |-- date: struct (nullable = true)
 |    |-- year: integer (nullable = false)
 |    |-- month: integer (nullable = false)
 |    |-- dayOfMonth: integer (nullable = false)
 |    |-- dayOfWeek: integer (nullable = false)
 |-- times: struct (nullable = true)
 |    |-- depTime: integer (nullable = false)
 |    |-- crsDepTime: integer (nullable = false)
 |    |-- arrTime: integer (nullable = false)
 |    |-- crsArrTime: integer (nullable = false)
 |    |-- actualElapsedTime: integer (nullable = false)
 |    |-- crsElapsedTime: integer (nullable = false)
 |    |-- airTime: integer (nullable = false)
 |    |-- arrDelay: integer (nullable = false)
 |    |-- depDelay: integer (nullable = false)
 |    |-- taxiIn: integer (nullable = false)
 |    |-- taxiOut: integer (nullable = false)
 |-- uniqueCarrier: string (nullable = true)
 |-- flightNum: integer (nullable = false)
 |-- tailNum: string (nullable = true)
 |-- origin: string (nullable = true)
 |-- dest: string (nullable = true)
 |-- di

show at cmd20.sc:3

1 / 1

show at cmd20.sc:3

1 / 1

...

show at cmd20.sc:4

1 / 1

show at cmd20.sc:4

1 / 1

+--------+-----------+-----------+---------+-----------+---------+----------+
|iataCode|airportName|   cityName|stateName|countryName| latitude| longitude|
+--------+-----------+-----------+---------+-----------+---------+----------+
|     00M|    Thigpen|Bay Springs|       MS|        USA|31.953764|-89.234505|
+--------+-----------+-----------+---------+-----------+---------+----------+



In [23]:
val flights_between_airpots = flights.select("origin","dest").groupBy("origin","dest").count()
.orderBy($"count".desc,$"origin",$"dest").show

show at cmd22.sc:2

2 / 2

show at cmd22.sc:2

200 / 200

+------+----+-----+
|origin|dest|count|
+------+----+-----+
|   SEA| SJC|    2|
|   SEA| PSP|    1|
+------+----+-----+



## Using DataFrame API: sql

In [27]:
import sqlContext.sql
flights.registerTempTable("flights")
airports.registerTempTable("airports")
val flights_between_airpots_sql = sql("""
select origin,dest, count(*) as count from flights
group by origin, dest
order by count desc,origin
""")
flights_between_airpots_sql.show()

show at cmd26.sc:9

2 / 2

show at cmd26.sc:9

200 / 200

+------+----+-----+
|origin|dest|count|
+------+----+-----+
|   SEA| SJC|    2|
|   SEA| PSP|    1|
+------+----+-----+



import sqlContext.sql

flights_between_airpots_sql: DataFrame = [origin: string, dest: string ... 1 more field]

In [29]:
flights_between_airpots_sql.cache
flights_between_airpots_sql.registerTempTable("fbasql")
val fba = flights_between_airpots_sql
val air = airports

res28_0: DataFrame = [origin: string, dest: string ... 1 more field]
fba: DataFrame = [origin: string, dest: string ... 1 more field]
air: DataFrame = [iataCode: string, airportName: string ... 5 more fields]